In [1]:
import time
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from konlpy.tag import Okt
import sys
import os
np.random.seed(1)
tf.set_random_seed(1)
os.environ["CUDA_VISIBLE_DEVICES"]="3"
okt = Okt()

/home/jhm9507/miniconda3/envs/jhm/lib/python3.6/site-packages/jpype/_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [2]:
f = open("./data/ratings_train.txt",encoding="UTF-8")
train_data = [line.split("\t")[1:] for line in f.read().splitlines()]
train_data = train_data[1:]
f = open("./data/ratings_test.txt",encoding="UTF-8")
test_data = [line.split("\t")[1:] for line in f.read().splitlines()]
test_data = test_data[1:]

In [3]:
print(len(train_data))
print(train_data[0])
print(len(test_data))
print(test_data[0])

150000
['아 더빙.. 진짜 짜증나네요 목소리', '0']
50000
['굳 ㅋ', '1']


In [4]:
for idx, data in enumerate(train_data):
    if data[0] == "":
        del train_data[idx]
for idx, data in enumerate(test_data):
    if data[0] == "":
        del test_data[idx]
print(len(train_data))
print(len(test_data))

149995
49997


In [5]:
# train_tokens = [["/".join(tag) for tag in okt.pos(sentence[0], norm=True, stem=True)] for sentence in train_data]
# test_tokens = [["/".join(tag) for tag in okt.pos(sentence[0], norm=True, stem=True)] for sentence in test_data]

In [6]:
# word2vec = Word2Vec(size=300, alpha=0.025, seed=1, sg=1, min_alpha=0.025)
# word2vec.build_vocab(tokens)
# for epoch in range(30):
#     word2vec.train(train_tokens, total_examples=word2vec.corpus_count, epochs=word2vec.iter)
#     word2vec.alpha -= 0.002
#     word2vec.min_alpha = word2vec.alpha
#     print("epochs: {}/30".format(epoch+1))
# word2vec.save("./model/word2vec.model")

In [7]:
word2vec = Word2Vec.load("./model/word2vec.model")

/home/jhm9507/miniconda3/envs/jhm/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
def embedding(word):
    if word in word2vec.wv.vocab:
        return word2vec.wv[word]
    else:
        return np.random.normal(size=(300))

In [9]:
# train_X = [[embedding(word) for word in sentence] for sentence in train_tokens]
# train_y = [[int(data[1])] for data in train_data]
# test_X = [[embedding(word) for word in sentence] for sentence in test_tokens]
# test_y = [[int(data[1])] for data in test_data]
# print(len(train_X))
# print(len(train_y))
# print(len(test_X))
# print(len(test_y))
# np.save("./data/train_X.npy",train_X)
# np.save("./data/train_y.npy",train_y)
# np.save("./data/test_X.npy",test_X)
# np.save("./data/test_y.npy",test_y)

In [10]:
train_X = np.load("./data/train_X.npy")
train_y = np.load("./data/train_y.npy")
test_X = np.load("./data/test_X.npy")
test_y = np.load("./data/test_y.npy")
print(len(train_X))
print(len(train_y))
print(len(test_X))
print(len(test_y))

149995
149995
49997
49997


In [11]:
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]

In [12]:
class BiLSTM:
    def __init__(self, input_size, layers, hidden_units, max_length, learning_rate, dropout_keep_prob):
        with tf.variable_scope("BiLSTM", reuse=tf.AUTO_REUSE):
            self.input_X = tf.placeholder(dtype=tf.float32, shape=[None, max_length, input_size], name="input_X")
            self.input_y = tf.placeholder(dtype=tf.float32, shape=[None,1], name="input_y")
            self.dropout_keep_prob = tf.placeholder(dtype=tf.float32, shape=(), name="dropout_keep_prob")
            self.seq_len = tf.placeholder(dtype=tf.int32, shape=[None], name="seq_len")
            
            self.output = self.bilstm(self.input_X, layers, hidden_units, dropout_keep_prob)
            
            self.loss = -tf.reduce_mean(self.input_y*tf.log(self.output) + (1-self.input_y)*tf.log(1-self.output))
            self.train = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
            self.pred = tf.equal(self.input_y, tf.cast(tf.to_int32(self.output >= 0.5),tf.float32))
            self.acc = tf.multiply(tf.reduce_mean(tf.cast(self.pred, tf.float32)), 100, name="acc")
            
            tf.summary.scalar(name="Loss", tensor=self.loss)
            tf.summary.scalar(name="Accuracy", tensor=self.acc)
            self.merge_graph = tf.summary.merge_all()
    
    def bilstm(self, X, layers, hidden_units, dropout_keep_prob):
        
        fw_cell_stack = []
        for i in range(layers):
            fw_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units, forget_bias=1.0, state_is_tuple=True)
            fw_cell = tf.nn.rnn_cell.DropoutWrapper(fw_cell, output_keep_prob=dropout_keep_prob)
            fw_cell_stack.append(fw_cell)
        fw_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(fw_cell_stack)
        
        bw_cell_stack = []
        for i in range(layers):
            bw_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units,forget_bias=1.0, state_is_tuple=True)
            bw_cell = tf.nn.rnn_cell.DropoutWrapper(bw_cell, output_keep_prob=dropout_keep_prob)
            bw_cell_stack.append(bw_cell)
        bw_lstm_cell = tf.nn.rnn_cell.MultiRNNCell(bw_cell_stack)
        
#         fw_lstm_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units)
#         fw_lstm_cell = tf.nn.rnn_cell.DropoutWrapper(fw_lstm_cell, output_keep_prob=dropout_keep_prob)
#         bw_lstm_cell =  tf.nn.rnn_cell.LSTMCell(hidden_units)
#         bw_lstm_cell = tf.nn.rnn_cell.DropoutWrapper(bw_lstm_cell, output_keep_prob=dropout_keep_prob)
        
        outputs,states = tf.nn.bidirectional_dynamic_rnn(cell_fw=fw_lstm_cell, cell_bw=bw_lstm_cell, inputs=X, dtype=tf.float32, sequence_length=self.seq_len)
        fw_output = outputs[0][:,-1]
        bw_output = outputs[1][:,0]
        
        W = tf.get_variable("W_output",shape=[2*hidden_units,1], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float32)
        b = tf.get_variable("b_output",shape=(), initializer=tf.zeros_initializer(), dtype=tf.float32)
#         output = tf.concat([states[0][1], states[1][1]], axis=1)
        output = tf.concat([fw_output, bw_output], axis=1)
        output = tf.nn.sigmoid(tf.matmul(output,W) + b, name="output")
        return output

In [13]:
input_size = 300
layers = 1
hidden_units = 128
batch_size = 32
dropout_keep_prob = 0.5
max_length = 100
learning_rate = 0.001
epochs = 3
n_class = 2

In [16]:
with tf.Graph().as_default():
    start = time.time()
    
    sess = tf.Session()
    print("Session started")
    model = BiLSTM(input_size, layers, hidden_units, max_length, learning_rate, dropout_keep_prob)
    print("Model was initialized")
    save_path = "./model/BiLSTM"
    n_batchs = (int)(len(train_X) / batch_size)
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter("./log", sess.graph)
    merge_graph = model.merge_graph
    global_steps = 0
    sess.run(tf.global_variables_initializer())
    
    def train_step(batch_X, batch_y, seq_lenght, steps, epoch):
        
        batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
        for b in range(batch_size):
            batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
        
        feed_dict = {
            model.input_X: batch_X_padded,
            model.input_y: batch_y,
            model.dropout_keep_prob: dropout_keep_prob,
            model.seq_len: seq_length
        }
        _, loss, acc = sess.run([model.train, model.loss, model.acc], feed_dict=feed_dict)
        if steps % 100 == 0:
            print("epochs: {}/{}, iters: {}, loss: {}, acc: {}".format(epoch+1,epochs,steps, loss, acc))
            summary = sess.run(merge_graph, feed_dict=feed_dict)
            writer.add_summary(summary, global_steps)
    for i in range(epochs):
        steps = 0
        for j in range(n_batchs):
            batch_X, batch_y = train_X[steps*batch_size:(steps+1)*batch_size], train_y[steps*batch_size:(steps+1)*batch_size]
            seq_length = train_seq_length[steps*batch_size:(steps+1)*batch_size]
            try:
                train_step(batch_X, batch_y, seq_length, steps, i)
            except ValueError:
                print(ValueError)
            except KeyboardInterrupt:
                saver.save(sess,save_path)
                print("Model was saved")
                sys.exit(1)
            steps += 1
            global_steps += 1
        saver.save(sess,save_path)
    saver.save(sess,save_path)
    print("Training finished")
    print("Model was saved")
    
    end = time.time()
    print("{} secs".format(end-start))

Session started
Model was initialized
epochs: 1/3, iters: 0, loss: 0.7241056561470032, acc: 46.875
epochs: 1/3, iters: 100, loss: 0.501392662525177, acc: 81.25
epochs: 1/3, iters: 200, loss: 0.6684079766273499, acc: 62.5
epochs: 1/3, iters: 300, loss: 0.5751429200172424, acc: 62.5
epochs: 1/3, iters: 400, loss: 0.5095279216766357, acc: 68.75
epochs: 1/3, iters: 500, loss: 0.5160624980926514, acc: 78.125
epochs: 1/3, iters: 600, loss: 0.32921114563941956, acc: 84.375
epochs: 1/3, iters: 700, loss: 0.3517100512981415, acc: 87.5
epochs: 1/3, iters: 800, loss: 0.49366921186447144, acc: 68.75
epochs: 1/3, iters: 900, loss: 0.40460699796676636, acc: 81.25
epochs: 1/3, iters: 1000, loss: 0.4962013363838196, acc: 68.75
epochs: 1/3, iters: 1100, loss: 0.3416743576526642, acc: 87.5
epochs: 1/3, iters: 1200, loss: 0.5161064863204956, acc: 71.875
epochs: 1/3, iters: 1300, loss: 0.32061558961868286, acc: 84.375
epochs: 1/3, iters: 1400, loss: 0.39668983221054077, acc: 84.375
epochs: 1/3, iters: 150

SystemExit: 1

In [38]:
test_len = len(test_X)

batch_size = 10000
batchs = int(test_len/batch_size)

with tf.Graph().as_default():
    sess = tf.Session()
    print("Session started")
    save_path = "./model/BiLSTM"
    saver = tf.train.import_meta_graph(save_path+".meta")
    saver.restore(sess, save_path)
    graph = tf.get_default_graph()
    print("Model was loaded")
    
    input_X = graph.get_tensor_by_name("BiLSTM/input_X:0")
    input_y = graph.get_tensor_by_name("BiLSTM/input_y:0")
    dropout_keep_prob = graph.get_tensor_by_name("BiLSTM/dropout_keep_prob:0")
    output = graph.get_tensor_by_name("BiLSTM/output:0")
    seq_len = graph.get_tensor_by_name("BiLSTM/seq_len:0")
    acc = graph.get_tensor_by_name("BiLSTM/acc:0")
    
    steps = 0
    acc_total = 0
    for b in range(batchs):
        batch_X, batch_y = test_X[steps*batch_size:(steps+1)*batch_size], test_y[steps*batch_size:(steps+1)*batch_size]
        batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
        for b in range(batch_size):
            batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
        seq_len_ = test_seq_length[steps*batch_size:(steps+1)*batch_size]
        feed_dict = {
            input_X: batch_X_padded,
            input_y: batch_y,
            dropout_keep_prob: 1.0,
            seq_len: seq_len_
        }
        acc_batch = sess.run(acc, feed_dict=feed_dict)
        acc_total += acc_batch
        steps += 1
        print("Batch {} 정확도: {}".format(steps,acc_batch))
    batch_X, batch_y = test_X[steps*batch_size:], test_y[steps*batch_size:]
    batch_size = len(batch_X)
    print("남은 데이터 수: {}".format(batch_size))
    batch_X_padded = np.zeros(shape=(batch_size,max_length,input_size))
    for b in range(batch_size):
        batch_X_padded[b,:len(batch_X[b])] = batch_X[b]
    seq_len_ = test_seq_length[steps*batch_size:(steps+1)*batch_size]
    feed_dict = {
        input_X: batch_X_padded,
        input_y: batch_y,
        dropout_keep_prob: 1.0,
        seq_len: seq_len_
    }
    acc_batch = sess.run(acc, feed_dict=feed_dict)
    acc_total += acc_batch
    steps += 1
    print("Batch {} 정확도: {}".format(steps,acc_batch))
    print("전체 정확도: {}".format(acc_total/(steps)))

Session started
INFO:tensorflow:Restoring parameters from ./model/BiLSTM
Model was loaded
Batch 1 정확도: 84.38999938964844
Batch 2 정확도: 84.33000183105469
Batch 3 정확도: 83.97000122070312
Batch 4 정확도: 84.20999908447266
남은 데이터 수: 9997
Batch 5 정확도: 75.1225357055664
전체 정확도: 82.40450744628906
